In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

import pandas as pd
import numpy as np
import random as rn
import copy
import string

!pip install emoji
import emoji

import re 
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score,cross_validate
from sklearn.metrics import make_scorer, accuracy_score, f1_score, precision_score, recall_score

In [ ]:
train_df = pd.read_csv('data/train_tweets.csv', names=['Tweets', 'Sentiment'])
val_df = pd.read_csv('data/test_tweets.csv', names=['Tweets', 'Sentiment'])

In [ ]:
train_df.replace({"olumsuz": "Negative", "olumlu": "Positive", "notr": "Neutral"}, inplace=True)
print('Train Sentiment classes', train_df['Sentiment'].unique())

In [ ]:
val_df.replace({"olumsuz": "Negative", "olumlu": "Positive", "notr": "Neutral"}, inplace=True)
print('Valid Sentiment classes', train_df['Sentiment'].unique())

In [7]:
train_df.groupby('Sentiment').size()

Sentiment
Negative    5511
Neutral     4658
Positive    3663
dtype: int64

In [8]:
val_df.groupby('Sentiment').size()

Sentiment
Negative    1377
Neutral     1164
Positive     916
dtype: int64

In [9]:
#@markdown This method intends to clean the given text \
#@markdown from graphical emojis, textual emojis, 
#@markdown mentions, and urls
import emoji
import re 
from tqdm import tqdm
import copy

textual_emoji_patterns = r"""(\:\)|\:\(|<3|\:\/|\:-\/|\:\||\:p|\(\:|\:\D)"""

mention_patterns = r"""(?:@[\w_]+)"""

url_patterns = r'http\//t.co\/[^\s]+|http\S+|www\S+'

multiple_spaces = r' {2,}'

tr_stopwords = stopwords.words('turkish')

def remove_textual_emoji(text):
    return re.sub(textual_emoji_patterns, '', text)

def remove_graphical_emoji(text): 
    return emoji.replace_emoji(text, '')

def remove_mentions(text):
    return re.sub(mention_patterns, '', text)

def remove_urls(text):
    return re.sub(url_patterns, '', text)

def remove_multiple_spaces(text):
    return re.sub(multiple_spaces, '', text)

cleaned_text = []
def clean_tweets(text):
    text = remove_textual_emoji(text)
    text = remove_graphical_emoji(text)
    text = remove_mentions(text)
    text = remove_urls(text)
    text = remove_multiple_spaces(text)
    text = text.rstrip()
    text = text.lower()
    stop_words = set(tr_stopwords)
    tokens = nltk.word_tokenize(text)
    tokens = [w.lower() for w in tokens]

    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    words = [w for w in words if not w in stop_words]
    
    cleaned_text.append(" ".join([w for w in words]))
    return cleaned_text


In [10]:
display(train_df["Tweets"].head(10))

0    Ulan Wifi'ye bağlıyım ben. Ona bağlıyken Turkc...
1    20 dk 1 GB internet 500 mb sadece kaşar turkce...
2    Ayrıca turkcell superonline reklamı kadar da k...
3                               Turkcell çok pahalı ya
4                  Turkcell Kaş'ta internetin cekmiyor
5    Turkcell'in Allah belası versin demek isterdim...
6                             Canın cehenneme turkcell
7    Turkcell yönetimini eline geçiren AKP hükümeti...
8    Turkcell şerefsizdir aksini iddia eden turkcel...
9         Turkcell'den elimi ayağımı çektiğim iyi oldu
Name: Tweets, dtype: object

In [ ]:
clean_tweet = []
for row in tqdm(train_df.itertuples(), desc="Clean the tweets", total=len(train_df)):
    doc = f"{clean_tweets(row.Tweets)}"
    clean_tweet.append(doc)
train_df["Tweets"] = clean_tweet
train_df["Tweets"].head(10)

Clean the tweets:  88%|████████▊ | 12109/13832 [01:21<00:18, 94.51it/s]